# Callbacks

In [38]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Hyperparameter Tunning

In [39]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [40]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [41]:
# Model is the full model w/o custom layers
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
    loss='sparse_categorical_crossentropy',  # Loss Function 
    metrics=['accuracy']  # Metrics / Accuracy
)

# Data Preprocess

In [42]:
os.listdir('../dataset/cifar/')

['labels.txt', 'test', 'train']

In [43]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]  # 보기용으로 데이터 갯수를 1000개로 제한
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [44]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [45]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [46]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [47]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [48]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [49]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [50]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [51]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Callbacks

In [52]:
start_time = datetime.now()

In [53]:
start_time

datetime.datetime(2021, 8, 5, 21, 52, 52, 359165)

In [54]:
end_time = datetime.now()

In [55]:
end_time

datetime.datetime(2021, 8, 5, 21, 52, 53, 342270)

In [56]:
end_time - start_time

datetime.timedelta(microseconds=983105)

In [57]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

In [58]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True,
    histogram_freq=1
)

https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/image_summaries.ipynb#scrollTo=IJNpyVyxbVtT

In [65]:
%tensorboard --logdir logs # --port 1111

Reusing TensorBoard on port 6006 (pid 17172), started 0:17:36 ago. (Use '!kill 17172' to kill it.)

## Training

http://localhost:6006

In [62]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

Epoch 1/10
31/31 [==============================] - 5s 120ms/step - loss: 2.2728 - accuracy: 0.1229 - val_loss: 2.2283 - val_accuracy: 0.1804
Epoch 2/10
31/31 [==============================] - 3s 102ms/step - loss: 2.1870 - accuracy: 0.1901 - val_loss: 2.1264 - val_accuracy: 0.2288
Epoch 3/10
31/31 [==============================] - 3s 102ms/step - loss: 2.1385 - accuracy: 0.1994 - val_loss: 2.1727 - val_accuracy: 0.1744
Epoch 4/10
31/31 [==============================] - 3s 100ms/step - loss: 2.0821 - accuracy: 0.2479 - val_loss: 2.0068 - val_accuracy: 0.2490
Epoch 5/10
31/31 [==============================] - 3s 102ms/step - loss: 2.0180 - accuracy: 0.2304 - val_loss: 1.9531 - val_accuracy: 0.2782
Epoch 6/10
31/31 [==============================] - 3s 100ms/step - loss: 1.9475 - accuracy: 0.2758 - val_loss: 1.9788 - val_accuracy: 0.2399
Epoch 7/10
31/31 [==============================] - 3s 99ms/step - loss: 1.9046 - accuracy: 0.2800 - val_loss: 1.8943 - val_accuracy: 0.2974
Epoch 8

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

### LambdaCallback

In [66]:
# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [67]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [68]:
# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [70]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

Epoch 1/10
31/31 [==============================] - 5s 114ms/step - loss: 1.7760 - accuracy: 0.3182 - val_loss: 1.7757 - val_accuracy: 0.3448


<ipython-input-66-7b8efe693ea4>:43: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)


Epoch 2/10
31/31 [==============================] - 3s 100ms/step - loss: 1.7277 - accuracy: 0.3595 - val_loss: 1.7584 - val_accuracy: 0.3407


<ipython-input-66-7b8efe693ea4>:43: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)


Epoch 3/10
31/31 [==============================] - 3s 100ms/step - loss: 1.7322 - accuracy: 0.3461 - val_loss: 1.7395 - val_accuracy: 0.3589


...

<ipython-input-66-7b8efe693ea4>:43: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)


Epoch 9/10
31/31 [==============================] - 3s 103ms/step - loss: 1.5760 - accuracy: 0.4173 - val_loss: 1.6748 - val_accuracy: 0.3921


<ipython-input-66-7b8efe693ea4>:43: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)


Epoch 10/10
31/31 [==============================] - 3s 104ms/step - loss: 1.5593 - accuracy: 0.4442 - val_loss: 1.6675 - val_accuracy: 0.3921


<ipython-input-66-7b8efe693ea4>:43: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)


## Expert

In [71]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [72]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [73]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [74]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [75]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [76]:
file_writer = tf.summary.create_file_writer(logdir)

In [77]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)

    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step+(step*epoch))
        tf.summary.scalar('train_loss', train_loss.result(), step=step+(step*epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step=step+(step*epoch))

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

KeyboardInterrupt: 

cmd창에서 tensorboard --logdir=./